In [ ]:
from openai import OpenAI
import json
import re
import subprocess


In [ ]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="", # Add your API key here
)

In [69]:
prompt = """What is the speed of a car if it starts from 0km/hr and accelerates at 5kms/hr after 2hrs.What is the capital of France?
"""

In [70]:
completion = client.chat.completions.create(
  model="google/gemini-2.0-pro-exp-02-05:free",
  messages=[
      {
        "role": "developer",
        "content": """You are an assistant with access to a Python execution environment.
        Your response **must** always be in JSON format.  
        - If the question can be solved with code, return:  
          ```json
          {"type": "code", "code": "<generated Python code>"}
          ```  
        - If it does not require code, return:  
          ```json
          {"type": "text", "answer": "<natural language response>"}
          ```  
        - If multiple questions with different types are asked, return 
        the non code response also in the code as a print statement.
        Prefer to generate code whenever possible.""",
      },
    {
    "role": "user",
    "content": prompt,
    }
  ]
)



In [71]:
print(completion.choices[0].message.content)

```json
{
    "type": "code",
    "code": "initial_speed = 0\nacceleration = 5\ntime = 2\n\nfinal_speed = initial_speed + (acceleration * time)\n\nprint(f'{final_speed=}')\nprint('The capital of France is Paris.')"
}
```


In [77]:
# Get the raw response from the LLM
raw_response = completion.choices[0].message.content
if raw_response.startswith("```json") and raw_response.endswith("```"):
    raw_response = re.sub(r"^```json|\n```$", "", raw_response).strip()

try:
    # Parse the JSON response
    response_data = json.loads(raw_response)
    # Check if it's code or text
    if response_data.get("type") == "code":
        generated_code = response_data.get("code", "")
        print("Generated Code:\n", generated_code)

        # Optionally, execute the code (be careful with untrusted input!)
        try:
            result = subprocess.run(
                ["python", "-c", generated_code], capture_output=True, text=True, timeout=5
            )
            execution_result = result.stdout.strip() if result.stdout else result.stderr.strip()
            print(execution_result)
        except Exception as e:
            execution_result = f"Error during execution: {str(e)}"
                # Send execution result back to LLM for final answer
        refinement_prompt = {
            "role": "user",
            "content": f"""You were asked a question :
            {prompt}
            You generated the following Python code to solve it:
            ```python
            {generated_code}
            ```
            The execution result was:
            ```
            {execution_result}
            ```
            Now, based on this, provide a final, well-informed answer to the user's original question.
            """,
        }
        refined_completion = client.chat.completions.create(
            model="google/gemini-2.0-pro-exp-02-05:free",
            messages=[refinement_prompt]
        )

        final_answer = refined_completion.choices[0].message.content.strip()
        print("\nFinal Answer:", final_answer)

    elif response_data.get("type") == "text":
        answer = response_data.get("answer", "")
        print("Answer:\n", answer)

    else:
        print("Unexpected response format:", response_data)

except json.JSONDecodeError as e:
    print("Error parsing JSON:", str(e))
    print("Raw response:", raw_response)


Generated Code:
 initial_speed = 0
acceleration = 5
time = 2

final_speed = initial_speed + (acceleration * time)

print(f'{final_speed=}')
print('The capital of France is Paris.')
final_speed=10
The capital of France is Paris.

Final Answer: The car's final speed is 10 km/hr.  The capital of France is Paris.


In [81]:
print("Prompt:\n", prompt)
print("Generated Code:\n", generated_code)
print("\nExecution Result:\n", execution_result)
print("\nFinal Answer:\n", final_answer)

Prompt:
 What is the speed of a car if it starts from 0km/hr and accelerates at 5kms/hr after 2hrs.What is the capital of France?

Generated Code:
 initial_speed = 0
acceleration = 5
time = 2

final_speed = initial_speed + (acceleration * time)

print(f'{final_speed=}')
print('The capital of France is Paris.')

Execution Result:
 final_speed=10
The capital of France is Paris.

Final Answer:
 The car's final speed is 10 km/hr.  The capital of France is Paris.
